### 패키지 설치


In [1]:
!pip install cloud-tpu-client==0.10 torch==1.12.0 https://storage.googleapis.com/tpu-pytorch/wheels/colab/torch_xla-1.12-cp37-cp37m-linux_x86_64.whl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 187.4 MB 34 kB/s 
     |████████████████████████████████| 776.3 MB 19 kB/s 
     |████████████████████████████████| 57 kB 3.0 MB/s 
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 1.12.11
    Uninstalling google-api-python-client-1.12.11:
      Successfully uninstalled google-api-python-client-1.12.11
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1+cu113
    Uninstalling torch-1.12.1+cu113:
      Successfully uninstalled torch-1.12.1+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.13.1+cu113 requires torch==1.12.1, but you have torch 1.12.0 which is incompatible.
torchtext 0.13.1 requires torch==1.12.1, but you have tor

In [2]:
!pip install transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 34.2 MB/s 
     |████████████████████████████████| 6.6 MB 50.5 MB/s 
     |████████████████████████████████| 120 kB 71.4 MB/s 


In [3]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 15.2 MB/s 
     |████████████████████████████████| 212 kB 61.8 MB/s 
     |████████████████████████████████| 115 kB 70.8 MB/s 
     |████████████████████████████████| 127 kB 71.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
earthengine-api 0.1.320 requires google-api-python-client<2,>=1.12.1, but you have google-api-python-client 1.8.0 which is incompatible.


### 시작

In [4]:
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertModel
from torchtext import datasets 
from torch.utils.data import TensorDataset, random_split
from torchtext.data.functional import to_map_style_dataset
from torch.utils.data import DataLoader
from torch.nn.functional import log_softmax
import random
import numpy as np
from torch.optim import Adam
from torch.nn.utils import clip_grad_norm_
from datasets import load_dataset
import warnings
from IPython.display import clear_output
warnings.filterwarnings("ignore")
RUN_EXAMPLES = True

In [5]:

# Acquires the default Cloud TPU core and moves the model to it
device = torch.device('cuda:0')

In [6]:
train, test = load_dataset('imdb', split=['train','test'])

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
labels=torch.tensor(train['label'])
txt=train['text']

In [8]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT tokenizer...


In [9]:
input_ids = []
attention_masks = []

# For every sentence...
for t in txt:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        t,         
                        truncation=True,             # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 64,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

# Print sentence 0, now as a list of IDs.
print('Original: ', txt[0])
print('Token IDs:', input_ids[0])

Original:  I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far b

In [10]:
from torch.utils.data import TensorDataset, random_split
np.random.seed(0)
# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(input_ids, attention_masks, labels)

# Create a 90-10 train-validation split.

# Calculate the number of samples to include in each set.
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

22,500 training samples
2,500 validation samples


In [11]:
from torch.utils.data import DataLoader

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 64

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            shuffle=True, 
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            shuffle=True,
            batch_size = batch_size # Evaluate with this batch size.
        )

In [12]:
class BertBinaryClassifier(nn.Module):
    def __init__(self, dropout=0.1):
        super(BertBinaryClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-uncased')

        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 2)
        self.softmax = nn.LogSoftmax()
    
    def forward(self, tokens, masks=None):
        temp = self.bert(tokens, attention_mask=masks)
        pooled_output=temp[1]
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        proba = log_softmax(linear_output, dim=-1)
        return proba

In [13]:
bert_clf = BertBinaryClassifier()
bert_clf = bert_clf.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [14]:
param_optimizer = list(bert_clf.softmax.named_parameters()) 
optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]

optimizer = Adam(bert_clf.parameters(), lr=3e-6)

In [17]:
import time
import datetime


def train(dataloader):
    bert_clf.train()
    train_loss = 0
    total_acc, total_count = 0, 0
    log_interval = 64
    start_time = time.time()
    for step_num, batch in enumerate(dataloader):
        optimizer.zero_grad()
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        logits = bert_clf(b_input_ids, b_input_mask)

        b_lables=b_labels.type(torch.LongTensor)
        batch_loss = criterion(logits, b_labels)
        batch_loss.backward()
        clip_grad_norm_(parameters=bert_clf.parameters(), max_norm=0.1)
        optimizer.step()
        
        total_acc += (logits.argmax(1) == b_labels).sum().item()
        total_count += b_labels.size(0)

        if step_num % log_interval == 0 and step_num > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, step_num, len(train_dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()



def evaluate(dataloader):
    bert_clf.eval()
    total_acc, total_count =0,0
    with torch.no_grad():
        for step_num, batch in enumerate(dataloader):

            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)

            logits = bert_clf(b_input_ids, b_input_mask)
            b_lables=b_labels.type(torch.LongTensor)
            loss = criterion(logits, b_labels)

            total_acc += (logits.argmax(1) == b_labels).sum().item()
            total_count += b_labels.size(0)
    return total_acc/total_count

        


In [18]:
torch.manual_seed(0)
criterion = torch.nn.CrossEntropyLoss()
EPOCHS=4
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = 0.6

for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(validation_dataloader)
    if total_accu is not None and total_accu > accu_val:
      scheduler.step()
    else:
       total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))
    print('-' * 59)

| epoch   1 |    64/  352 batches | accuracy    0.517
| epoch   1 |   128/  352 batches | accuracy    0.666
| epoch   1 |   192/  352 batches | accuracy    0.742
| epoch   1 |   256/  352 batches | accuracy    0.757
| epoch   1 |   320/  352 batches | accuracy    0.766
-----------------------------------------------------------
| end of epoch   1 | time: 245.04s | valid accuracy    0.791 
-----------------------------------------------------------
| epoch   2 |    64/  352 batches | accuracy    0.800
| epoch   2 |   128/  352 batches | accuracy    0.800
| epoch   2 |   192/  352 batches | accuracy    0.801
| epoch   2 |   256/  352 batches | accuracy    0.808
| epoch   2 |   320/  352 batches | accuracy    0.803
-----------------------------------------------------------
| end of epoch   2 | time: 263.87s | valid accuracy    0.818 
-----------------------------------------------------------
| epoch   3 |    64/  352 batches | accuracy    0.822
| epoch   3 |   128/  352 batches | accura

In [20]:
test_input_ids = []
test_attention_masks = []
txt_test=test['text']
labels_test=torch.tensor(test['label'])

# For every sentence...
for t in txt_test:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        t,         
                        truncation=True,             # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 64,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    test_input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    test_attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
test_input_ids = torch.cat(test_input_ids, dim=0)
test_attention_masks = torch.cat(test_attention_masks, dim=0)

testset = TensorDataset(test_input_ids, test_attention_masks, labels_test)



In [22]:
test_dataloader = DataLoader(
            testset,  # The training samples.
            shuffle=True, 
            batch_size = 64 # Trains with this batch size.
        )

In [23]:
accu_test = evaluate(test_dataloader)
print('test accuracy {:8.3f}'.format(accu_test))

test accuracy    0.828
